This Jupyter notebook assumes that the R kernel for Jupyter (IRkernel) has been installed; see
https://irkernel.github.io/installation/

# R code for logistic regression analysis of bar fractions

## Requirements

This notebook is meant to be run within the full **s4g_barfractions** repository, including the associated data files.

In addition, this notebook requires the following R packages:
   * [survey](https://cran.r-project.org/package=survey)

## Setup

In [1]:
library(survey)

ERROR: Error in library(survey): there is no package called ‘survey’


Set the following so that it points to the directory with the (text) data files:

In [ ]:
basedir <- "./data/"

## Weighted logistic regression for Sample 1: log(M_star) alone

Logistic regression for fraction of galaxies with bars as a function of stellar mass $\log (M_{\star} / M_{\odot})$, using S4G galaxies in Sample 1 (spirals at $D \leq 25$ Mpc) with stellar
masses between $\log M_{\star} = 8.5$ and 11, with $V/V_{\rm max}$ weighting to account for S4G angular diameter limit.

Load data into table and then Survey-package design object

In [ ]:
ff <- paste(basedir, "barpresence_vs_logmstar_for_R_w25_m8.5-11.txt", sep="")
logmstarBarWTable <- read.table(ff, header=TRUE)
logmstarBarWDesign <- svydesign(ids=~0, data=logmstarBarWTable, weights=~weight)
length(logmstarBarWTable$bar)

Standard linear logistic regression: bar fraction versus log of stellar mass

In [ ]:
logMstarWFit1 <- svyglm(bar ~ logmstar, design=logmstarBarWDesign, family=quasibinomial)
summary(logMstarWFit1)

Quadratic linear logistic regression: bar fraction versus log of stellar mass + square of same

In [ ]:
logMstarWFit2 <- svyglm(bar ~ logmstar + I(logmstar^2), design=logmstarBarWDesign, family=quasibinomial)
summary(logMstarWFit2)

### Comparison of AIC values

In [ ]:
AIC(logMstarWFit1)
AIC(logMstarWFit2)

In [ ]:
747.73 - 762.586

#### Summary

Since the quadratic fit has $\Delta$AIC $= -14.9$ relative to the linear fit, it is clearly preferred.

## Weighted logistic regression for Sample 1: f(bar) vs log(M_star) and g-r

Same as previous section, but now we do logistic regression versus both stellar mass and $g - r$ color, using a subsample
of Sample 1 galaxies with color data.

In [ ]:
ff <- paste(basedir, "barpresence_vs_logmstar-gmr_for_R_w25.txt", sep="")
logmstargmrBarWTable <- read.table(ff, header=TRUE)
gmrBarWDesign <- svydesign(ids=~0, data=logmstargmrBarWTable, weights=~weight)
length(logmstargmrBarWTable$bar)

### Linear fit of $f_{\rm bar}$ vs just $g - r$

In [ ]:
gmrWFit_gmr <- svyglm(bar ~ gmr, design=gmrBarWDesign, family=quasibinomial)
summary(gmrWFit_gmr)

### Fit vs just logMstar for same sample: linear, then quadratic

In [ ]:
# same sample, vs logmstar (linear) only
gmrWFit_logmstar <- svyglm(bar ~ logmstar, design=gmrBarWDesign, family=quasibinomial)
summary(gmrWFit_logmstar)

In [ ]:
# same sample, vs logmstar (quadratic) only
gmrWFit_logmstar2 <- svyglm(bar ~ logmstar + I(logmstar^2), design=gmrBarWDesign, family=quasibinomial)
summary(gmrWFit_logmstar2)

### Finally, fit vs logMstar (quadratic) *and* g-r

In [ ]:
gmrWFit_gmrlogmstar2 <- svyglm(bar ~ logmstar + I(logmstar^2) + gmr, design=gmrBarWDesign, family=quasibinomial)
summary(gmrWFit_gmrlogmstar2)

### Comparison of AIC values

In [ ]:
AIC(gmrWFit_gmr)
AIC(gmrWFit_logmstar)
AIC(gmrWFit_logmstar2)
AIC(gmrWFit_gmrlogmstar2)

#### Summary

Best fit from AIC standpoint is quadratic logMstar (*without* $g - r$) -- note that its AIC is actually *lower*
than the AIC for the quadratic logMstar + $g - r$ fit.

## Weighted logistic regression for Sample 1: f(bar) vs log(M_star) and log(f_gas)

Same as previous section, but now we do logistic regression versus both log of stellar mass and log of gas mass ratio $f{\rm gas} = M_{\rm HI} / M_{\star}$, using a subsample
of Sample 1 galaxies with H I data.

In [ ]:
basedir <- "/Users/erwin/Documents/Working/Projects/Project_BarSizes/"
ff <- paste(basedir, "barpresence_vs_logmstar-logfgas_for_R_w25.txt", sep="")
logMstarfgasBarWTable <- read.table(ff, header=TRUE)
logMstarfgasBarWDesign <- svydesign(ids=~0, data=logMstarfgasBarWTable, weights=~weight)
length(logMstarfgasBarWTable$bar)

### Fit vs just log(f_gas)

In [ ]:
logMstarlogfgasWFit_fgas <- svyglm(bar ~ logfgas, design=logMstarfgasBarWDesign, family=quasibinomial)
summary(logMstarlogfgasWFit_fgas)

### Fit vs just logMstar: linear, then quadratic

In [ ]:
logMstarlogfgasWFit_logmstar <- svyglm(bar ~ logmstar, design=logMstarfgasBarWDesign, family=quasibinomial)
summary(logMstarlogfgasWFit_logmstar)

In [ ]:
logMstarlogfgasWFit_logmstar2 <- svyglm(bar ~ logmstar + I(logmstar^2), design=logMstarfgasBarWDesign, family=quasibinomial)
summary(logMstarlogfgasWFit_logmstar2)

### Finally, fit vs logMstar (quadratic) *and* log(f_gas)

In [ ]:
logMstarlogfgasWFit_fgaslogmstar2 <- svyglm(bar ~ logmstar + I(logmstar^2) + logfgas, design=logMstarfgasBarWDesign, family=quasibinomial)
summary(logMstarlogfgasWFit_fgaslogmstar2)

### Comparison of AIC values

In [ ]:
AIC(logMstarlogfgasWFit_fgas)
AIC(logMstarlogfgasWFit_logmstar)
AIC(logMstarlogfgasWFit_logmstar2)
AIC(logMstarlogfgasWFit_fgaslogmstar2)

#### Summary

The quadratic fit using logMstar (without log f_gas) is clearly the best model.